In [1]:
import os
import spotipy
from spotipy.oauth2 import SpotifyOAuth
from spotipy.oauth2 import SpotifyClientCredentials

import pandas as pd
import json

import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *

In [2]:
#extract credentials from environment variabes and instantiate session
auth_manager = SpotifyClientCredentials()
sp = spotipy.Spotify(auth_manager=auth_manager)

In [3]:
#grab playlists for user
myPlaylists = sp.user_playlists("ben.doan4366")

#extract tracks for gym and cooking playlists
gymPlaylist = sp.playlist_tracks("1Vu8EHOlNPBFVBxmKtGHlr")
cookingPlaylist = sp.playlist_tracks("5myx61jwlbF2iMDVWNK0Ps")

In [4]:
#sample raw repsonse from Spotify
gymPlaylist

{'href': 'https://api.spotify.com/v1/playlists/1Vu8EHOlNPBFVBxmKtGHlr/tracks?offset=0&limit=100&additional_types=track',
 'items': [{'added_at': '2016-09-12T21:45:52Z',
   'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/ben.doan4366'},
    'href': 'https://api.spotify.com/v1/users/ben.doan4366',
    'id': 'ben.doan4366',
    'type': 'user',
    'uri': 'spotify:user:ben.doan4366'},
   'is_local': False,
   'primary_color': None,
   'track': {'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6oMuImdp5ZcFhWP0ESe6mG'},
       'href': 'https://api.spotify.com/v1/artists/6oMuImdp5ZcFhWP0ESe6mG',
       'id': '6oMuImdp5ZcFhWP0ESe6mG',
       'name': 'Migos',
       'type': 'artist',
       'uri': 'spotify:artist:6oMuImdp5ZcFhWP0ESe6mG'}],
     'available_markets': ['AD',
      'AE',
      'AG',
      'AL',
      'AM',
      'AO',
      'AR',
      'AT',
      'AU',
      'AZ',
      'BA',
      'BB',
     

In [88]:
#function to transition raw json response from Spotify to dataframe of artist and so

def prepArtistStagingTables(raw_playlist_dictionary):
    
    #trim meta data and extract track field from raw dictionary
    cleaned_playlist_dict = [sub['track'] for sub in raw_playlist_dictionary['items']]
    
    #generate raw dataframe
    playlist_df_raw = pd.read_json(json.dumps(cleaned_playlist_dict))
    
    #extract artist column from pd
    artist_df = playlist_df_raw.loc[:,["name","artists"]].explode("artists")
    
    #extract artist dictionary values into series, and recast to dataframe 
    artist_fields_expanded = artist_df["artists"].apply(pd.Series)
    artist_fields_expanded.rename(columns={"name":"artist_name"}, inplace=True)
    
    #concat recasted dataframe to original dataframe with song name
    #artist_df_final = pd.concat([artist_df, artist_fields_expanded], axis=1)
    artist_df_final = pd.concat([artist_df, artist_fields_expanded], axis=1).loc[:,["name", "id", "artist_name", "type", "uri"]]
    artist_df_final.rename(columns={"name":"song_name"}, inplace=True)
    artist_df_final.rename(columns={"id":"artist_id"}, inplace=True)
    
    return artist_df_final

In [89]:
def prepAlbumStagingTables(raw_playlist_dictionary):
    #trim meta data and extract item field from raw dictionary
    cleaned_playlist_dict = [sub['track'] for sub in raw_playlist_dictionary['items']]
    
    #generate raw dataframe
    playlist_df_raw = pd.read_json(json.dumps(cleaned_playlist_dict))
    
    
    #extract artist column from pd
    album_df = playlist_df_raw.loc[:,["name", "album"]]
    album_df_serialized = album_df["album"].apply(pd.Series)
    album_df_serialized.rename(columns={"name":"album_name"}, inplace=True)
    album_df_serialized.rename(columns={"id":"album_id"}, inplace=True)
    album_df_serialized.rename(columns={"uri":"album_uri"}, inplace=True)

    #extract artist dictionary values into series, and recast to dataframe 
    album_with_artists_df = album_df_serialized.explode("artists")
    album_with_artists_df_serialized = album_with_artists_df["artists"].apply(pd.Series)
    album_with_artists_df_serialized.rename(columns={"name":"artist_name"}, inplace=True)
    album_with_artists_df_serialized.rename(columns={"id":"artist_id"}, inplace=True)

    
    #concat recasted dataframe to original dataframe with song name
    album_df_final = pd.concat([album_with_artists_df_serialized, album_df_serialized, album_df], axis=1)
    album_df_final.rename(columns={"name":"song_name"}, inplace=True)
    album_df_final.rename(columns={"id":"album_id"}, inplace=True)
    
    return album_df_final.loc[:,["album_name", "album_id", "artist_name", "artist_id","song_name","release_date", "album_uri"]]

In [103]:
def generate_song_nodes(raw_playlist_dictionary):
    cleaned_playlist_dict = [sub['track'] for sub in raw_playlist_dictionary['items']]
    playlist_df_raw = pd.read_json(json.dumps(cleaned_playlist_dict))
    
    song_nodes = playlist_df_raw.loc[:,["name", "id", "uri","duration_ms","explicit","popularity"]]
    song_nodes.rename(columns={"name":"song_name"}, inplace=True)
    song_nodes.rename(columns={"id":"song_id"}, inplace=True)
    song_nodes.drop_duplicates(["song_id"], inplace=True)
    
    return song_nodes

In [114]:
def generate_artist_nodes(artist_df):
    artist_nodes = artist_df.loc[:,["artist_name", "artist_id"]]
    artist_nodes.drop_duplicates(["artist_id"], inplace=True)
    
    return artist_nodes

In [118]:
def generate_album_nodes(album_df):
    album_nodes = album_df.loc[:,["album_name", "album_id"]]
    album_nodes.drop_duplicates(["album_id"], inplace=True)
    
    return album_nodes

In [131]:
cooking_playlist_artists_df = prepArtistStagingTables(cookingPlaylist)
gym_playlist_artists_df = prepArtistStagingTables(gymPlaylist)

full_playlist_artists_df = pd.concat([cooking_playlist_artists_df, gym_playlist_artists_df])
full_playlist_artists_df

,song_name,artist_id,artist_name,type,uri
0,Come Over,7GN9PivdemQRKjDt4z5Zv8,The Internet,artist,spotify:artist:7GN9PivdemQRKjDt4z5Zv8
1,By the Poolside,73dudJ9j0HStIhJDU8MjMI,Feng Suave,artist,spotify:artist:73dudJ9j0HStIhJDU8MjMI
2,Keep The Faith,3cIXmCH7iNcslTbwrwS7zy,Moon Boots,artist,spotify:artist:3cIXmCH7iNcslTbwrwS7zy
2,Keep The Faith,1NrFTpkB0RvbVLYl0p5Xvc,Nic Hanson,artist,spotify:artist:1NrFTpkB0RvbVLYl0p5Xvc
3,Westside,6f96znq79wvlknKHHHhtTW,Jarreau Vandal,artist,spotify:artist:6f96znq79wvlknKHHHhtTW
...,...,...,...,...,...
97,Opps (with Yugen Blakrok),68kEuyFKyqrdQQLLsmiatm,Vince Staples,artist,spotify:artist:68kEuyFKyqrdQQLLsmiatm
97,Opps (with Yugen Blakrok),3kv1Edgn5HlEWCuEKr1Y9x,Yugen Blakrok,artist,spotify:artist:3kv1Edgn5HlEWCuEKr1Y9x
98,44 More,4xRYI6VqpkE3UwrDrAZL8L,Logic,artist,spotify:artist:4xRYI6VqpkE3UwrDrAZL8L
99,Achoo,6f9bmfkqiYbhCtdAZsv7KI,Keith Ape,artist,spotify:artist:6f9bmfkqiYbhCtdAZsv7KI


In [135]:
cooking_playlist_albums_df = prepAlbumStagingTables(cookingPlaylist)
gym_playlist_albums_df = prepAlbumStagingTables(gymPlaylist)

full_album_playlist_df = pd.concat([cooking_playlist_albums_df, gym_playlist_albums_df])
full_album_playlist_df

,album_name,album_id,artist_name,artist_id,song_name,release_date,album_uri
0,Hive Mind,27ThgFMUAx3MXLQ297DzWF,The Internet,7GN9PivdemQRKjDt4z5Zv8,Come Over,2018-07-20,spotify:album:27ThgFMUAx3MXLQ297DzWF
1,Feng Suave,4TXLnG7URez82o8xfZif2h,Feng Suave,73dudJ9j0HStIhJDU8MjMI,By the Poolside,2017-10-04,spotify:album:4TXLnG7URez82o8xfZif2h
2,Keep The Faith,6XBv9s0hqnoX7lV6wydBeV,Moon Boots,3cIXmCH7iNcslTbwrwS7zy,Keep The Faith,2017-06-27,spotify:album:6XBv9s0hqnoX7lV6wydBeV
2,Keep The Faith,6XBv9s0hqnoX7lV6wydBeV,Nic Hanson,1NrFTpkB0RvbVLYl0p5Xvc,Keep The Faith,2017-06-27,spotify:album:6XBv9s0hqnoX7lV6wydBeV
3,Westside,1ZyxE4rXSfAZwqd6PO7l1N,Jarreau Vandal,6f96znq79wvlknKHHHhtTW,Westside,2018-07-31,spotify:album:1ZyxE4rXSfAZwqd6PO7l1N
...,...,...,...,...,...,...,...
97,Black Panther The Album Music From And Inspire...,3pLdWdkj83EYfDN6H2N8MR,Kendrick Lamar,2YZyLoL8N0Wb9xBt1NhZWg,Opps (with Yugen Blakrok),2018-02-09,spotify:album:3pLdWdkj83EYfDN6H2N8MR
97,Black Panther The Album Music From And Inspire...,3pLdWdkj83EYfDN6H2N8MR,SZA,7tYKF4w9nC0nq9CsPZTHyP,Opps (with Yugen Blakrok),2018-02-09,spotify:album:3pLdWdkj83EYfDN6H2N8MR
98,44 More,6xEahv0rPP5PHJh4kpQ6jT,Logic,4xRYI6VqpkE3UwrDrAZL8L,44 More,2018-02-23,spotify:album:6xEahv0rPP5PHJh4kpQ6jT
99,Achoo,7A1YvHtwUCd4EjhJBd3zTo,Keith Ape,6f9bmfkqiYbhCtdAZsv7KI,Achoo,2017-10-03,spotify:album:7A1YvHtwUCd4EjhJBd3zTo


In [129]:
gym_song_nodes = generate_song_nodes(gymPlaylist)
cooking_playlist_nodes = generate_song_nodes(cookingPlaylist)

all_song_nodes = pd.concat([gym_song_nodes, cooking_playlist_nodes])
all_song_nodes

,song_name,song_id,uri,duration_ms,explicit,popularity
0,Fight Night,2n5gVJ9fzeX2SSWlLQuyS9,spotify:track:2n5gVJ9fzeX2SSWlLQuyS9,216247,True,68
1,Pipe It Up,6eBrlbv2HMYcldwjoMWIrC,spotify:track:6eBrlbv2HMYcldwjoMWIrC,206266,True,53
2,pick up the phone,4GXl8l1MfZPf2GvpyRPJBf,spotify:track:4GXl8l1MfZPf2GvpyRPJBf,252262,True,68
3,"Floyd Mayweather (feat. Travis Scott, Gucci Ma...",5ALc7rbru6QOLGodVSDocc,spotify:track:5ALc7rbru6QOLGodVSDocc,358506,True,56
4,With Them,0tISnxqgVmxqhVghsTi2Rr,spotify:track:0tISnxqgVmxqhVghsTi2Rr,197746,True,58
...,...,...,...,...,...,...
95,Think,5TocQE0LESbiqk6sd3Y5Hm,spotify:track:5TocQE0LESbiqk6sd3Y5Hm,231927,False,33
96,All Yours,76S0dx36oCAMgqfXlVUQeZ,spotify:track:76S0dx36oCAMgqfXlVUQeZ,173586,False,51
97,DaNcing in a RoOm,2zTjF6OGxklU5Ulv7iTf4R,spotify:track:2zTjF6OGxklU5Ulv7iTf4R,208605,False,0
98,Enginn eins og þú,3bzP8lZJWmuhpdnfDP1rTq,spotify:track:3bzP8lZJWmuhpdnfDP1rTq,181226,False,40


In [138]:
all_artist_nodes = generate_artist_nodes(full_playlist_artists_df)
all_artist_nodes

,artist_name,artist_id
0,The Internet,7GN9PivdemQRKjDt4z5Zv8
1,Feng Suave,73dudJ9j0HStIhJDU8MjMI
2,Moon Boots,3cIXmCH7iNcslTbwrwS7zy
2,Nic Hanson,1NrFTpkB0RvbVLYl0p5Xvc
3,Jarreau Vandal,6f96znq79wvlknKHHHhtTW
...,...,...
96,Jay Rock,28ExwzUQsvgJooOI0X1mr3
96,James Blake,53KwLdlmrlCelAZMaLVZqU
97,Yugen Blakrok,3kv1Edgn5HlEWCuEKr1Y9x
99,Keith Ape,6f9bmfkqiYbhCtdAZsv7KI


In [139]:
all_album_nodes = generate_album_nodes(full_album_playlist_df)
all_album_nodes

,album_name,album_id
0,Hive Mind,27ThgFMUAx3MXLQ297DzWF
1,Feng Suave,4TXLnG7URez82o8xfZif2h
2,Keep The Faith,6XBv9s0hqnoX7lV6wydBeV
3,Westside,1ZyxE4rXSfAZwqd6PO7l1N
4,Anything In Return,3xDRuOqakukb1SjHQG4WWc
...,...,...
90,Culture II,7fd7SEK25VS3gJAUgSwL6y
93,Cozy Tapes Vol. 2: Too Cozy,0qr1Fvi1haEDWVbFtekZLb
94,Black Panther The Album Music From And Inspire...,3pLdWdkj83EYfDN6H2N8MR
98,44 More,6xEahv0rPP5PHJh4kpQ6jT


In [152]:
full_album_playlist_df

,album_name,album_id,artist_name,artist_id,song_name,release_date,album_uri
0,Hive Mind,27ThgFMUAx3MXLQ297DzWF,The Internet,7GN9PivdemQRKjDt4z5Zv8,Come Over,2018-07-20,spotify:album:27ThgFMUAx3MXLQ297DzWF
1,Feng Suave,4TXLnG7URez82o8xfZif2h,Feng Suave,73dudJ9j0HStIhJDU8MjMI,By the Poolside,2017-10-04,spotify:album:4TXLnG7URez82o8xfZif2h
2,Keep The Faith,6XBv9s0hqnoX7lV6wydBeV,Moon Boots,3cIXmCH7iNcslTbwrwS7zy,Keep The Faith,2017-06-27,spotify:album:6XBv9s0hqnoX7lV6wydBeV
2,Keep The Faith,6XBv9s0hqnoX7lV6wydBeV,Nic Hanson,1NrFTpkB0RvbVLYl0p5Xvc,Keep The Faith,2017-06-27,spotify:album:6XBv9s0hqnoX7lV6wydBeV
3,Westside,1ZyxE4rXSfAZwqd6PO7l1N,Jarreau Vandal,6f96znq79wvlknKHHHhtTW,Westside,2018-07-31,spotify:album:1ZyxE4rXSfAZwqd6PO7l1N
...,...,...,...,...,...,...,...
97,Black Panther The Album Music From And Inspire...,3pLdWdkj83EYfDN6H2N8MR,Kendrick Lamar,2YZyLoL8N0Wb9xBt1NhZWg,Opps (with Yugen Blakrok),2018-02-09,spotify:album:3pLdWdkj83EYfDN6H2N8MR
97,Black Panther The Album Music From And Inspire...,3pLdWdkj83EYfDN6H2N8MR,SZA,7tYKF4w9nC0nq9CsPZTHyP,Opps (with Yugen Blakrok),2018-02-09,spotify:album:3pLdWdkj83EYfDN6H2N8MR
98,44 More,6xEahv0rPP5PHJh4kpQ6jT,Logic,4xRYI6VqpkE3UwrDrAZL8L,44 More,2018-02-23,spotify:album:6xEahv0rPP5PHJh4kpQ6jT
99,Achoo,7A1YvHtwUCd4EjhJBd3zTo,Keith Ape,6f9bmfkqiYbhCtdAZsv7KI,Achoo,2017-10-03,spotify:album:7A1YvHtwUCd4EjhJBd3zTo


In [153]:
all_song_nodes

,song_name,song_id,uri,duration_ms,explicit,popularity
0,Fight Night,2n5gVJ9fzeX2SSWlLQuyS9,spotify:track:2n5gVJ9fzeX2SSWlLQuyS9,216247,True,68
1,Pipe It Up,6eBrlbv2HMYcldwjoMWIrC,spotify:track:6eBrlbv2HMYcldwjoMWIrC,206266,True,53
2,pick up the phone,4GXl8l1MfZPf2GvpyRPJBf,spotify:track:4GXl8l1MfZPf2GvpyRPJBf,252262,True,68
3,"Floyd Mayweather (feat. Travis Scott, Gucci Ma...",5ALc7rbru6QOLGodVSDocc,spotify:track:5ALc7rbru6QOLGodVSDocc,358506,True,56
4,With Them,0tISnxqgVmxqhVghsTi2Rr,spotify:track:0tISnxqgVmxqhVghsTi2Rr,197746,True,58
...,...,...,...,...,...,...
95,Think,5TocQE0LESbiqk6sd3Y5Hm,spotify:track:5TocQE0LESbiqk6sd3Y5Hm,231927,False,33
96,All Yours,76S0dx36oCAMgqfXlVUQeZ,spotify:track:76S0dx36oCAMgqfXlVUQeZ,173586,False,51
97,DaNcing in a RoOm,2zTjF6OGxklU5Ulv7iTf4R,spotify:track:2zTjF6OGxklU5Ulv7iTf4R,208605,False,0
98,Enginn eins og þú,3bzP8lZJWmuhpdnfDP1rTq,spotify:track:3bzP8lZJWmuhpdnfDP1rTq,181226,False,40


In [156]:
album_edges_staging = full_album_playlist_df.merge(all_song_nodes, on=["song_name"])
album_edges_staging

,album_name,album_id,artist_name,artist_id,song_name,release_date,album_uri,song_id,uri,duration_ms,explicit,popularity
0,Hive Mind,27ThgFMUAx3MXLQ297DzWF,The Internet,7GN9PivdemQRKjDt4z5Zv8,Come Over,2018-07-20,spotify:album:27ThgFMUAx3MXLQ297DzWF,2hPNuVVSV1tqiD2uPlfehz,spotify:track:2hPNuVVSV1tqiD2uPlfehz,321853,True,57
1,Feng Suave,4TXLnG7URez82o8xfZif2h,Feng Suave,73dudJ9j0HStIhJDU8MjMI,By the Poolside,2017-10-04,spotify:album:4TXLnG7URez82o8xfZif2h,4XTWiowLNAWfdc3YNq59VU,spotify:track:4XTWiowLNAWfdc3YNq59VU,217116,False,0
2,Keep The Faith,6XBv9s0hqnoX7lV6wydBeV,Moon Boots,3cIXmCH7iNcslTbwrwS7zy,Keep The Faith,2017-06-27,spotify:album:6XBv9s0hqnoX7lV6wydBeV,33RKGVnB3zWdvHwTpq4ApA,spotify:track:33RKGVnB3zWdvHwTpq4ApA,258034,False,40
3,Keep The Faith,6XBv9s0hqnoX7lV6wydBeV,Nic Hanson,1NrFTpkB0RvbVLYl0p5Xvc,Keep The Faith,2017-06-27,spotify:album:6XBv9s0hqnoX7lV6wydBeV,33RKGVnB3zWdvHwTpq4ApA,spotify:track:33RKGVnB3zWdvHwTpq4ApA,258034,False,40
4,Westside,1ZyxE4rXSfAZwqd6PO7l1N,Jarreau Vandal,6f96znq79wvlknKHHHhtTW,Westside,2018-07-31,spotify:album:1ZyxE4rXSfAZwqd6PO7l1N,4rU2eED2YPnmmrVmLlaYVI,spotify:track:4rU2eED2YPnmmrVmLlaYVI,179629,False,27
...,...,...,...,...,...,...,...,...,...,...,...,...
220,Black Panther The Album Music From And Inspire...,3pLdWdkj83EYfDN6H2N8MR,Kendrick Lamar,2YZyLoL8N0Wb9xBt1NhZWg,Opps (with Yugen Blakrok),2018-02-09,spotify:album:3pLdWdkj83EYfDN6H2N8MR,7bUcBztfGqO7cSI2gMZeCI,spotify:track:7bUcBztfGqO7cSI2gMZeCI,180893,True,59
221,Black Panther The Album Music From And Inspire...,3pLdWdkj83EYfDN6H2N8MR,SZA,7tYKF4w9nC0nq9CsPZTHyP,Opps (with Yugen Blakrok),2018-02-09,spotify:album:3pLdWdkj83EYfDN6H2N8MR,7bUcBztfGqO7cSI2gMZeCI,spotify:track:7bUcBztfGqO7cSI2gMZeCI,180893,True,59
222,44 More,6xEahv0rPP5PHJh4kpQ6jT,Logic,4xRYI6VqpkE3UwrDrAZL8L,44 More,2018-02-23,spotify:album:6xEahv0rPP5PHJh4kpQ6jT,6iyp6udVpLuq3MCUulZT4h,spotify:track:6iyp6udVpLuq3MCUulZT4h,188613,True,0
223,Achoo,7A1YvHtwUCd4EjhJBd3zTo,Keith Ape,6f9bmfkqiYbhCtdAZsv7KI,Achoo,2017-10-03,spotify:album:7A1YvHtwUCd4EjhJBd3zTo,2huH9q26JmfR0BzV4xKzny,spotify:track:2huH9q26JmfR0BzV4xKzny,244000,True,0


In [157]:
artist_edges_staging = full_artist_playlist_df.merge(all_song_nodes, on=["song_name"])
artist_edges_staging

,song_name,artist_id,artist_name,type,uri_x,song_id,uri_y,duration_ms,explicit,popularity
0,Come Over,7GN9PivdemQRKjDt4z5Zv8,The Internet,artist,spotify:artist:7GN9PivdemQRKjDt4z5Zv8,2hPNuVVSV1tqiD2uPlfehz,spotify:track:2hPNuVVSV1tqiD2uPlfehz,321853,True,57
1,By the Poolside,73dudJ9j0HStIhJDU8MjMI,Feng Suave,artist,spotify:artist:73dudJ9j0HStIhJDU8MjMI,4XTWiowLNAWfdc3YNq59VU,spotify:track:4XTWiowLNAWfdc3YNq59VU,217116,False,0
2,Keep The Faith,3cIXmCH7iNcslTbwrwS7zy,Moon Boots,artist,spotify:artist:3cIXmCH7iNcslTbwrwS7zy,33RKGVnB3zWdvHwTpq4ApA,spotify:track:33RKGVnB3zWdvHwTpq4ApA,258034,False,40
3,Keep The Faith,1NrFTpkB0RvbVLYl0p5Xvc,Nic Hanson,artist,spotify:artist:1NrFTpkB0RvbVLYl0p5Xvc,33RKGVnB3zWdvHwTpq4ApA,spotify:track:33RKGVnB3zWdvHwTpq4ApA,258034,False,40
4,Westside,6f96znq79wvlknKHHHhtTW,Jarreau Vandal,artist,spotify:artist:6f96znq79wvlknKHHHhtTW,4rU2eED2YPnmmrVmLlaYVI,spotify:track:4rU2eED2YPnmmrVmLlaYVI,179629,False,27
...,...,...,...,...,...,...,...,...,...,...
348,Opps (with Yugen Blakrok),68kEuyFKyqrdQQLLsmiatm,Vince Staples,artist,spotify:artist:68kEuyFKyqrdQQLLsmiatm,7bUcBztfGqO7cSI2gMZeCI,spotify:track:7bUcBztfGqO7cSI2gMZeCI,180893,True,59
349,Opps (with Yugen Blakrok),3kv1Edgn5HlEWCuEKr1Y9x,Yugen Blakrok,artist,spotify:artist:3kv1Edgn5HlEWCuEKr1Y9x,7bUcBztfGqO7cSI2gMZeCI,spotify:track:7bUcBztfGqO7cSI2gMZeCI,180893,True,59
350,44 More,4xRYI6VqpkE3UwrDrAZL8L,Logic,artist,spotify:artist:4xRYI6VqpkE3UwrDrAZL8L,6iyp6udVpLuq3MCUulZT4h,spotify:track:6iyp6udVpLuq3MCUulZT4h,188613,True,0
351,Achoo,6f9bmfkqiYbhCtdAZsv7KI,Keith Ape,artist,spotify:artist:6f9bmfkqiYbhCtdAZsv7KI,2huH9q26JmfR0BzV4xKzny,spotify:track:2huH9q26JmfR0BzV4xKzny,244000,True,0


In [160]:
all_album_nodes.to_csv("../data/album_nodes.csv")
all_artist_nodes.to_csv("../data/artist_nodes.csv")
all_song_nodes.to_csv("../data/song_nodes.csv")

album_edges_staging.to_csv("../data/album_edges.csv")
artist_edges_staging.to_csv("../data/artist_edges.csv")